## Chassis Example Notebooks
Welcome to the examples section for [Chassis](https://chassis.ml), which contains notebooks that auto-containerize models built using the most common machine learning (ML) frameworks. 

#### What is Chassis?
Chassis allows you to automatically create a Docker container from your model code and push that container image to a Docker registry. All you need is your model loaded into memory and a few lines of Chassis code! Our example bank is here to provide reference examples for many common ML frameworks.  

Can't find the framework you are looking for or need help? Fork this repository and open a PR, or list the desired framework in a new issue. We're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [1]:
import time
import chassisml
import cv2
import torch
import getpass
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision import transforms

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')

docker hub username········
docker hub password········


## Prepare model and data transform
Initialize anything here that should persist across inference runs

In [3]:
model = models.segmentation.deeplabv3_resnet50(pretrained=True)
model.eval()

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])    

sem_classes = [
    '__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]
sem_class_to_idx = {cls: idx for (idx, cls) in enumerate(sem_classes)}

device = 'cpu'

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to C:\Users\BradleyMunday/.cache\torch\hub\checkpoints\deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [4]:
def process(input_bytes):
    
    # preprocess
    decoded = cv2.imdecode(np.frombuffer(input_bytes, np.uint8), -1)
    img_t = transform(decoded)
    batch_t = torch.unsqueeze(img_t, 0).to(device)
    
    # run inference
    predictions = model(batch_t)["out"]
    
    # postprocess
    normalized_masks = torch.nn.functional.softmax(predictions, dim=1)
    
    # limit to dog class due to mask size 
    dog_mask = normalized_masks[0][sem_class_to_idx["dog"]]
    inference_result = {
        "classSegmentationMasks": [
            {"class": "dog", "mask": dog_mask.detach().cpu().numpy().tolist()}
        ]
    }

    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }
    
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [5]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com")

## Create and test Chassis model
* Requires `process_fn` defined above

In [6]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/dog.jpg'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":{"classSegmentationMasks":[{"class":"dog","mask":[[0.00019701893324963748,0.00019701893324963748,0.00019701893324963748,0.00019701893324963748,0.00018195260781794786,0.00015858585538808256,0.00013821314496453851,0.00012045283801853657,0.00010497090988792479,9.147616947302595e-05,7.971412560436875e-05,6.946271605556831e-05,7.103740790626034e-05,7.692223880439997e-05,8.329427510034293e-05,9.019394929055125e-05,9.766464063432068e-05,0.00010575393389444798,0.00011451253521954641,0.00012399630213622004,0.00013123091775923967,0.00013851434050593525,0.00014620134606957436,0.00015431421343237162,0.00016287653124891222,0.00017191321239806712,0.00018145029025617987,0.000191336905118078,0.0001984032423933968,0.0002057303790934384,0.00021332784672267735,0.00022120556968729943,0.00022937401081435382,0.0002378434146521613,0.00024662577197887003,0.00025397626450285316,0.00025487635866738856,0.00025577849010005593,0.0002566829789429903,0.0002575896796770394,0.0002584983012638986,0.

In [7]:
# manually construct conda environment to pass to Chassis job

# NOTE: if you define your environment manually like this, the "chassisml" package must be included within the pip depedencies
env = {
    "name": "chassis-env",
    "channels": ['conda-forge'],
    "dependencies": [
        "python=3.8.5",
        {
            "pip": [
                "numpy",
                "chassisml",
                "torch",
                "torchvision",
                "opencv-python-headless"
            ] 
        }
    ]
}

## Publish model to Docker
Need to provide model name, model version, and Dockerhub credentials

In [8]:
MODEL_NAME = "PyTorch Deeplab ResNet50 Semantic Segmentation"
start_time = time.time()
response = chassis_model.publish(
    model_name=MODEL_NAME,
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    conda_env=env
)

Starting build job... Ok!


In [9]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)
end_time = time.time()
if final_status['status']['succeeded'] == 1:
    print("Job Completed in {} minutes.\n\nView your new container image here: https://hub.docker.com/repository/docker/{}/{}".format(round((end_time-start_time)/60, 5), dockerhub_user, "-".join(MODEL_NAME.lower().split(" "))))
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))

Job Completed in 12.24141 minutes.

View your new container image here: https://hub.docker.com/repository/docker/bmunday131/pytorch-deeplab-resnet50-semantic-segmentation
